In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import homogeneity_score
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [2]:
df_metrics = pd.DataFrame(columns=['ARI_Louvain','ARI_kmeans','ARI_HC',
                                   'AMI_Louvain','AMI_kmeans','AMI_HC',
                                   'Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC'])

In [3]:
workdir = './output_new/'
path_fm = os.path.join(workdir,'feature_matrices/')
path_clusters = os.path.join(workdir,'clusters/')
path_metrics = os.path.join(workdir,'metrics/')
os.system('mkdir -p '+path_clusters)
os.system('mkdir -p '+path_metrics)

0

In [4]:
metadata = pd.read_csv('../input/metadata.tsv',sep='\t',index_col=0)
num_clusters = len(np.unique(metadata['label']))
print(num_clusters)

13


In [5]:
files = [x for x in os.listdir(path_fm) if x.startswith('fm')]
len(files)

18

In [6]:
files

['fm_Control.rds',
 'fm_Control_new.rds',
 'fm_BROCKMAN.rds',
 'fm_BROCKMAN_new.rds',
 'fm_Cusanovich2018.rds',
 'fm_Cusanovich2018_new.rds',
 'fm_chromVAR_kmers2.rds',
 'fm_chromVAR_kmers2_new.rds',
 'fm_chromVAR_motifs2.rds',
 'fm_chromVAR_motifs2_new.rds',
 'fm_GeneScoring2.rds',
 'fm_GeneScoring2_new.rds',
 'fm_Cicero2.rds',
 'fm_Cicero2_new.rds',
 'fm_SnapATAC.rds',
 'fm_SnapATAC_new.rds',
 'fm_SCRAT2.rds',
 'fm_SCRAT2_new.rds']

In [7]:
def getNClusters(adata,n_cluster,range_min=0,range_max=3,max_steps=20):
    this_step = 0
    this_min = float(range_min)
    this_max = float(range_max)
    while this_step < max_steps:
        print('step ' + str(this_step))
        this_resolution = this_min + ((this_max-this_min)/2)
        sc.tl.louvain(adata,resolution=this_resolution)
        this_clusters = adata.obs['louvain'].nunique()
        
        print('got ' + str(this_clusters) + ' at resolution ' + str(this_resolution))
        
        if this_clusters > n_cluster:
            this_max = this_resolution
        elif this_clusters < n_cluster:
            this_min = this_resolution
        else:
            return(this_resolution, adata)
        this_step += 1
    
    print('Cannot find the number of clusters')
    print('Clustering solution from last iteration is used:' + str(this_clusters) + ' at resolution ' + str(this_resolution))

In [8]:
for file in files:
    file_split = file[:-4].split('_')
    method = file_split[1]
    if(len(file_split)>2):
        method = method + '_'+''.join(file_split[2:]).replace('2','_pca')
    print(method)

    pandas2ri.activate()
    readRDS = robjects.r['readRDS']
    df_rds = readRDS(os.path.join(path_fm,file))
    fm_mat = pandas2ri.ri2py(robjects.r['data.frame'](robjects.r['as.matrix'](df_rds)))
    fm_mat.fillna(0,inplace=True)
    fm_mat.columns = metadata.index
    
    adata = sc.AnnData(fm_mat.T)
    adata.var_names_make_unique()
    adata.obs = metadata.loc[adata.obs.index,]
    df_metrics.loc[method,] = ""
    #Louvain
    sc.pp.neighbors(adata, n_neighbors=15,use_rep='X')
#     sc.tl.louvain(adata)
    getNClusters(adata,n_cluster=num_clusters)
    #kmeans
    kmeans = KMeans(n_clusters=num_clusters, random_state=2019).fit(adata.X)
    adata.obs['kmeans'] = pd.Series(kmeans.labels_,index=adata.obs.index).astype('category')
    #hierachical clustering
    hc = AgglomerativeClustering(n_clusters=num_clusters).fit(adata.X)
    adata.obs['hc'] = pd.Series(hc.labels_,index=adata.obs.index).astype('category')
    #clustering metrics
    
    #adjusted rank index
    ari_louvain = adjusted_rand_score(adata.obs['label'], adata.obs['louvain'])
    ari_kmeans = adjusted_rand_score(adata.obs['label'], adata.obs['kmeans'])
    ari_hc = adjusted_rand_score(adata.obs['label'], adata.obs['hc'])
    #adjusted mutual information
    ami_louvain = adjusted_mutual_info_score(adata.obs['label'], adata.obs['louvain'],average_method='arithmetic')
    ami_kmeans = adjusted_mutual_info_score(adata.obs['label'], adata.obs['kmeans'],average_method='arithmetic')   
    ami_hc = adjusted_mutual_info_score(adata.obs['label'], adata.obs['hc'],average_method='arithmetic')
    #homogeneity
    homo_louvain = homogeneity_score(adata.obs['label'], adata.obs['louvain'])
    homo_kmeans = homogeneity_score(adata.obs['label'], adata.obs['kmeans'])
    homo_hc = homogeneity_score(adata.obs['label'], adata.obs['hc'])

    df_metrics.loc[method,['ARI_Louvain','ARI_kmeans','ARI_HC']] = [ari_louvain,ari_kmeans,ari_hc]
    df_metrics.loc[method,['AMI_Louvain','AMI_kmeans','AMI_HC']] = [ami_louvain,ami_kmeans,ami_hc]
    df_metrics.loc[method,['Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC']] = [homo_louvain,homo_kmeans,homo_hc] 
    adata.obs[['louvain','kmeans','hc']].to_csv(os.path.join(path_clusters ,method + '_clusters.tsv'),sep='\t')

Control


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 33 at resolution 1.5
step 1
got 23 at resolution 0.75
step 2
got 16 at resolution 0.375
step 3
got 12 at resolution 0.1875
step 4
got 13 at resolution 0.28125
Control_new


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 26 at resolution 1.5
step 1
got 19 at resolution 0.75
step 2
got 14 at resolution 0.375
step 3
got 12 at resolution 0.1875
step 4
got 13 at resolution 0.28125
BROCKMAN


...

step 0
got 25 at resolution 1.5
step 1
got 20 at resolution 0.75
step 2
got 17 at resolution 0.375
step 3
got 14 at resolution 0.1875
step 4
got 12 at resolution 0.09375
step 5
got 14 at resolution 0.140625
step 6
got 14 at resolution 0.1171875
step 7
got 13 at resolution 0.10546875
SCRAT2


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 27 at resolution 1.5
step 1
got 15 at resolution 0.75
step 2
got 9 at resolution 0.375
step 3
got 11 at resolution 0.5625
step 4
got 13 at resolution 0.65625
SCRAT2_new


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 28 at resolution 1.5
step 1
got 15 at resolution 0.75
step 2
got 10 at resolution 0.375
step 3
got 13 at resolution 0.5625


In [9]:
df_metrics.to_csv(path_metrics+'clustering_scores.csv')

In [10]:
df_metrics

,ARI_Louvain,ARI_kmeans,ARI_HC,AMI_Louvain,AMI_kmeans,AMI_HC,Homogeneity_Louvain,Homogeneity_kmeans,Homogeneity_HC
Control,0.332837,0.0310206,0.0323049,0.548613,0.17679,0.18392,0.54098,0.145575,0.146171
Control_new,0.322932,0.0359537,0.0633727,0.568016,0.287147,0.341796,0.550649,0.211888,0.2627
BROCKMAN,0.229773,0.108314,0.0950577,0.402846,0.245153,0.216923,0.403183,0.246948,0.218872
BROCKMAN_new,0.350076,0.208846,0.216572,0.517632,0.377216,0.389107,0.519581,0.378841,0.386687
Cusanovich2018,0.314571,0.312622,0.394549,0.588781,0.532839,0.579267,0.562294,0.520058,0.575403
Cusanovich2018_new,0.397886,0.343153,0.381751,0.614461,0.566933,0.579881,0.595005,0.549324,0.574297
chromVAR_kmers_pca,0.393402,0.353105,0.330552,0.57713,0.537296,0.523283,0.570369,0.529479,0.519517
chromVAR_kmers_pcanew,0.270334,0.321985,0.275228,0.532833,0.509146,0.465866,0.506553,0.501988,0.458794
chromVAR_motifs_pca,0.252594,0.237495,0.222042,0.475341,0.383546,0.387681,0.449016,0.380229,0.371064
chromVAR_motifs_pcanew,0.252875,0.211211,0.172324,0.408515,0.316595,0.32332,0.394885,0.315727,0.314548
